recommendation sys


In [1]:
import numpy as np
import pandas as pd

In [2]:
brand_data = pd.read_csv("brand_data.csv")
influencer_data = pd.read_csv("influencer_dataset.csv")

In [3]:
brand_data.head()

,brand_id,brand_name,product_category,audience_age,audience_gender,audience_location
0,201.0,BeautyCo,Cosmetics,18-25,Female,Madhya Pradesh
1,202.0,FitLife,Fitness Equipment,25-34,Male,Delhi
2,203.0,TechGear,Gadgets,18-24,Male,India
3,204.0,FashionX,Clothing,18-30,Female,India
4,205.0,TravelNow,Travel Services,25-40,All,"Goa, Jammu and Kashmir"


In [4]:
influencer_data.head()

,Influencer_ID,Influencer Name,Handle,Followers,Posts,Likes,Category,Sentiment,Location,Engagement Rate
0,1,Liam Brown,@sophia.w,662606,445,22863,Gaming,Positive,US,3.450467
1,2,Isabella Moore,@isabella.m,198089,595,46116,Travel,Neutral,Italy,23.280445
2,3,Olivia Clark,@isabella.m,581816,679,15803,Music,Positive,India,2.716151
3,4,John Smith,@olivia.c,560060,702,19824,Fitness,Negative,Canada,3.539621
4,5,Emma Taylor,@jane.doe,273159,672,6424,Photography,Negative,Brazil,2.351744
